# Exp 06: Demonstrate Numba
Numba, https://numba.pydata.org/, is a high-performance jit compiler.  
What that means is that functions using NumPy can be optimized into machine code.  
Is that faster?


In [1]:
# standard libraries
from time import perf_counter_ns
import time

In [2]:
# external libraries
import numpy as np
import pandas as pd

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *

In [3]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)

...loading words into a dataframe...
...query execution took: 1.25 seconds...
...loading word groups into a dataframe...
...query execution took: 1.25 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


In [4]:
ls_df = build_letter_selector_df(df = wg_df, 
                                 ls_nchar=3, letter_selector_col_name='letter_selector',
                                 letter_selector_id_col_name='letter_selector_id')
ls_df = get_ls_index(df = ls_df)

...loading the letter dictionary...


In [5]:
ls_df.head()

,letter_selector,ls_count,ls_nchar_iter,ls_nchar,letter_selector_id,ls_index
0,a,2,3,1,0,"[True, False, False, False, False, False, Fals..."
1,ae,1,3,2,1,"[True, False, False, False, True, False, False..."
2,ai,1,3,2,2,"[True, False, False, False, False, False, Fals..."
3,b,1,3,1,3,"[False, True, False, False, False, False, Fals..."
4,ba,4,3,2,4,"[True, True, False, False, False, False, False..."


In [6]:
# load the total number of anagrams
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH,
                                             db_name=rc.DB_NAME)

...query execution took: 0.0 seconds...


In [7]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,letter_selector,n_records
0,A,a,1,a,0,0,a,a,1,a,1
1,aa,aa,2,a,1,1,a,a,1,a,1
2,aal,aal,3,a,2,2,al,la,2,la,1
3,aalii,aalii,5,a,3,3,ail,lai,1,lai,1
4,aam,aam,3,a,4,4,am,ma,2,ma,1


In [8]:
col_names = ['letter_selector', 'letter_selector_id']
wg_df = pd.merge(left = wg_df, right = ls_df[col_names])

In [9]:
ls_id_wg_id, ls_index_array = build_ls_index_arrays(wg_df=wg_df, ls_df = ls_df)

In [10]:
# import numba and types
from numba import njit, int8, int32

In [11]:
# decorate a function that encapsulates several NumPy operations
# notably, numba cannot be used with np.all(), so we have to get creative
# with a replacement that involves a loop. NumPy's strength is that so many
# operations work along the dimensions of arrays.
@njit
def build_output_wg_id_list(temp_wg_id_list:np.ndarray, ls_wchar_matrix:np.ndarray, temp_wg_id:int) -> np.ndarray:
    # numba doesn't have the equivalent of np.all which means I need 
    # to implement a work around.
    rows, cols = ls_wchar_matrix.shape
    temp_matrix = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0    
    zero_list = np.zeros(shape = rows, dtype=np.bool)
    for i in range(rows):
        zero_list[i] = temp_matrix[i, :].sum() == cols    
    return temp_wg_id_list[zero_list]

In [12]:
# Build functions to contain operations. In theory, the just-in-time compilation
# of Numba should make this faster. 
@njit
def numbafunc01(wchar_matrix:np.ndarray, outcome_indices:np.ndarray):
    # this is the sub-matrix from which to query
    return wchar_matrix[outcome_indices, :]

@njit
def numbafunc02(word_group_id_list:np.ndarray, outcome_indices:np.ndarray):
    return word_group_id_list[outcome_indices]

@njit
def numbafunc03(ls_id_wg_id:np.ndarray, ls_row_id:int):
    return ls_id_wg_id[ls_id_wg_id[:, 0] == ls_row_id, 1]

@njit
def numbafunc04(temp_wg_id_list:np.ndarray, curr_wg_id:np.ndarray):
    return np.where(temp_wg_id_list == curr_wg_id)[0][0]

@njit
def numbafunc05(ls_wchar_matrix:np.ndarray, temp_wg_id:int):
    return ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]


In [13]:
# finally, decorate the format_output_list() function.
@njit
def format_output_list(outcome_word_id_list: np.ndarray, wg_id: int) -> np.ndarray:
    output_list = np.zeros(
        shape=(outcome_word_id_list.shape[0], 2), dtype=np.int32)

    # update the output list with the word_id_list - these are from/parent words
    output_list[:, 0] = outcome_word_id_list

    # update with the word_id - this is the to/child word
    output_list[:, 1] = wg_id

    return output_list


In [14]:
# run it!
# we need to compile the function, so, let's run this once, toss those results,
# and then actually run it
for time_counter in range(2):
    if time_counter == 0:
        slices = 1
        print('...running once to compile code...')
    else:
        slices = None
        print('...running all letter selectors...')


    run_start_time=perf_counter_ns()
    # create the output list
    output_list = np.full(shape = (n_possible_anagrams, 2), fill_value=-1, dtype=np.int32)
    output_time_list = []

    # start counting
    anagram_pair_count = 0

    for ls_row_id, ls_row in enumerate(ls_index_array[:slices, :]):    
        if ls_row_id % 100 == 0:
            print(ls_row_id)
        start_time = perf_counter_ns()    
                
        ##
        # SUBSET THE wchar_matrix by column selector
        ##    
        outcome_indices = np.all(wchar_matrix[:, ls_row] >= 1, axis=1)
        
        # leaving the lines in to show what's been decorated with Numba
        #ls_wchar_matrix = wchar_matrix[outcome_indices, :]
        ls_wchar_matrix = numbafunc01(wchar_matrix = wchar_matrix, outcome_indices = outcome_indices)
            
        # this is the list of word group ids that correspond to the word group ids
        # in the ls_wchar_matrix
        #temp_wg_id_list = word_group_id_list[outcome_indices]
        temp_wg_id_list = numbafunc02(word_group_id_list=word_group_id_list,
                                    outcome_indices=outcome_indices)
        
        # this is the number of word groups that meet certain criteria. 
        # for example, words that feature the letters: 'buc'    
        n_search_space = temp_wg_id_list.shape[0]    
    
        #curr_wg_id_list = ls_id_wg_id[ls_id_wg_id[:, 0] == ls_row_id, 1]
        curr_wg_id_list = numbafunc03(ls_id_wg_id=ls_id_wg_id, ls_row_id=ls_row_id)    
        
        for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):    
                
            # get different word group ids?
            #temp_wg_id = np.where(temp_wg_id_list == curr_wg_id)[0][0]
            temp_wg_id = numbafunc04(temp_wg_id_list=temp_wg_id_list, curr_wg_id=curr_wg_id)        
                    
            #outcome_word_id_list = temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0, axis = 1)]        
            temp_matrix = numbafunc05(ls_wchar_matrix=ls_wchar_matrix, temp_wg_id=temp_wg_id)
            outcome_word_id_list = temp_wg_id_list[np.all(a = temp_matrix >= 0, axis = 1)]
                    
            n_from_words = outcome_word_id_list.shape[0]
            
            if n_from_words > 0:
                outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list, wg_id=curr_wg_id)
                            
                # enumerate the from/parent words
                new_anagram_pair_count = anagram_pair_count + n_from_words
                
                output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list

                # update the anagram pair count
                anagram_pair_count = new_anagram_pair_count

        curr_time = calc_time(time_start=start_time, round_digits=8)
        output_time_list.append([ls_row_id, n_search_space, curr_time])

    print('...time to find parent/child word relationships')
    time_proc = calc_time(time_start=run_start_time, round_digits=4)
    compute_elapsed_time(seconds=time_proc)
    print('...truncating output list...')
    output_indices = np.all(output_list >= 0, axis=1)
    output_list = output_list[output_indices,]
    print(output_list.shape)
    time_proc = calc_time(time_start=run_start_time, round_digits=4)
    compute_elapsed_time(seconds=time_proc)

...running once to compile code...
0
...time to find parent/child word relationships
Hours: 0 | minutes: 0 | seconds: 7.9557
...truncating output list...
(176230, 2)
Hours: 0 | minutes: 0 | seconds: 10.1699
...running all letter selectors...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
...time to find parent/child word relationships
Hours: 0 | minutes: 1 | seconds: 58.3989
...truncating output list...
(73218235, 2)
Hours: 0 | minutes: 2 | seconds: 2.2738


In [15]:
# current technique: 1 minute, 19 seconds
# using Numba: 1 minute, 50 seconds. Not faster!

In [16]:
# check the counters, just to confirm
from_word_counter, to_word_counter = build_counters(output_list=output_list)

In [17]:
# the number of from word groups: should be 26
print(from_word_counter[746])
print(to_word_counter[746])

26
329


In [18]:
# the take away: Numba - in this example - is not faster. 